In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
    groq_api_key='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'
)

response = llm.invoke("Who landed first on the moon?")
print(response.content)

Neil Armstrong landed first on the moon on July 20, 1969, as part of the Apollo 11 mission. He stepped out of the lunar module Eagle and onto the moon's surface, famously declaring, "That's one small step for man, one giant leap for mankind."


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.jpmorgan.com/us/en/students/programs/cadp-summer-analyst")
page_data = loader.load().pop().page_content
print(page_data)


USER_AGENT environment variable not set, consider setting it to identify your requests.








Corporate Analyst Development Program Internship | JPMorgan Chase & Co.  



































We no longer support this browser.  Using a supported browser will provide a better experience.
Please update your browser.



Close browser message







Skip to Main Content









Open Navigation








Students programs






Work with us






Grow with us






How we hire






Careers















Submit to search 














                        Jobs, Student Programs & Internships 
                        












Internship

Corporate Analyst 
Development Program
















Creating the next generation of leaders
Immerse yourself alongside industry experts, refining processes and identifying opportunities that support our global businesses. In this dynamic team environment, you'll make meaningful contributions while developing core business skills in process improvement, project management, and analytics. These experiences will equip you wit

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    (page_data)
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data' :page_data})
print(res.content)

```python
import json
import re

# Assuming page_data is a string containing the scraped text
page_data = """
Job Title: Software Engineer
Experience: 5-7 years
Skills: Python, Java, C++, JavaScript
Description: We are looking for a skilled software engineer to join our team.

Job Title: Data Scientist
Experience: 3-5 years
Skills: Python, R, SQL, Machine Learning
Description: We are looking for a data scientist to analyze and interpret complex data.

Job Title: DevOps Engineer
Experience: 2-4 years
Skills: AWS, Docker, Kubernetes, Jenkins
Description: We are looking for a devops engineer to manage our cloud infrastructure.
"""

# Regular expression pattern to match job postings
pattern = r"""
Job Title: (?P<role>[^ ]+)
Experience: (?P<experience>[^ ]+)
Skills: (?P<skills>[^ ]+)
Description: (?P<description>[^ ]+)
"""

# Find all matches of the pattern in the page data
matches = re.findall(pattern, page_data, re.DOTALL | re.VERBOSE)

# Convert matches to JSON
json_data = []
for match i

In [16]:
from langchain_core.output_parsers import JsonOutputParser
import re

llm_output = res.content

match = re.search(r"\{\s*\".*?\"\s*:\s*\".*?\"\s*\}", llm_output, re.DOTALL)

if not match:
    raise ValueError("No valid JSON object found in LLM output.")

json_str = match.group(0)

parser = JsonOutputParser()
parsed = parser.parse(json_str)

print(parsed)


{'role': 'Software Engineer', 'experience': '5-7 years', 'skills': 'Python, Java, C++, JavaScript', 'description': 'We are looking for a skilled software engineer to join our team.'}


In [17]:
type(parsed)


dict

In [19]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )


In [33]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [35]:
job

{'role': 'Software Engineer',
 'experience': '5-7 years',
 'skills': 'Python, Java, C++, JavaScript',
 'description': 'We are looking for a skilled software engineer to join our team.'}

In [32]:
job = parsed
job['skills']

'Python, Java, C++, JavaScript'

In [8]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

NameError: name 'llm' is not defined